# Example of using the advertrain library

In [6]:
import torch
from torch import nn, optim
from pathlib import Path
import data_all
from robustAI.advertrain.models import ConvNet, ResNet, ConvNetDropblock, ResNetDropblock
from robustAI.advertrain.training.classical_training import ClassicalTraining
from robustAI.advertrain.training.adversarial_training import AdversarialTraining
from robustAI.advertrain.training.autoattack_training import AutoAttackTraining
from robustAI.advertrain.training.fire_training import FIRETraining
from robustAI.advertrain.training.trades_training import TRADESTraining
from robustAI.advertrain.transforms import DataTransformations
import sys
import os
import argparse

In [25]:
# Set here the path to the directory containing the folder c00 of the example dataset. it can be downloaded
# here http://minio-storage.apps.confianceai-public.irtsysx.fr/ml-models/robust-ml-dataset.zip

ROOT_DIR = "/home/jovyan/Maturation/robust-ai/examples/dataset/c00" 

DEVICE = "cuda:1"
ARCHITECTURE = "resnetdropblock"
LEARNING_RATE = 1e-4
MODELS_PATH = "results/"

In [26]:
transformer = DataTransformations()
train_transform = transformer.get_train_transforms()
test_transform = transformer.get_test_transforms()

In [27]:
path_data = "learning_data/"

train_dataloader1, val_dataloader1 = data_all.get_train_dataloader(
    ROOT_DIR,
    path_data+"train_dataset_all.csv",
    path_data+"val_dataset_all.csv",
    batch_size=16,
    train_transform=train_transform,
    test_transform=test_transform,
)

In [28]:
if ARCHITECTURE == "convnet":
    model = ConvNet(DEVICE)
elif ARCHITECTURE == "resnet":
    model = ResNet(DEVICE)
elif ARCHITECTURE == "convnetdropblock":
    model = ConvNetDropblock(DEVICE)
elif ARCHITECTURE == "resnetdropblock":
    model = ResNetDropblock(DEVICE)
else:
    raise ValueError("Unknown architecture")

In [29]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1, 1]).to(DEVICE))

In [30]:
if not torch.cuda.is_available() or DEVICE =="cpu":
        print("ERROR : Learning should be done on a system with GPU")
        sys.exit()
        
print(torch.cuda.current_device())
print("GPU device used ", DEVICE)

0
GPU device used  cuda:1


## Train class

### ClassicalTraining

In [31]:
METHODE="vanilla"

In [32]:
trainer = ClassicalTraining(model, optimizer, criterion, DEVICE)

### AdversarialTraining

In [33]:
METHODE = "adversarial_training"
EPSILON = 8 / 255

In [34]:
trainer = AdversarialTraining(model, optimizer, criterion, DEVICE, EPSILON)

### AutoAttackTraining

In [35]:
METHODE = "autoattack_training"
EPSILON = 8 / 255

In [36]:
trainer = AutoAttackTraining(model, optimizer, criterion, DEVICE, "ce", EPSILON)

### FireTraining

In [37]:
METHODE = "fire_training"
EPSILON = 8 / 255
BETA = 1

In [38]:
trainer = FIRETraining(model, optimizer, DEVICE, EPSILON, BETA)

### TradesTraining

In [39]:
METHODE = "trades_training"
EPSILON = 8 / 255
BETA = 1

In [40]:
trainer = TRADESTraining(model, optimizer, DEVICE, EPSILON, BETA)

## Fit the model

In [41]:
path = os.path.join(MODELS_PATH,
                    f"{ARCHITECTURE}"+"_"+f"{METHODE}")

if not os.path.exists(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass

print(f"Using '{path}' directory")

trainer.fit(epochs=2, train_dataloader=train_dataloader1, val_dataloader=val_dataloader1, patience=2, checkpoint=path)

Using 'results/resnetdropblock_trades_training' directory


Epochs 1/2 : Training:   0%|          | 0/750 [00:00<?, ?it/s]/home/jovyan/Maturation/env-robustml/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Validation: 100%|██████████| 33/33 [00:08<00:00,  3.94it/s]


Epoch 1/2
Train Loss:  0.702, Acc:  0.660, Recall:  0.625, Precision:  0.666,F1 Score:  0.645
Validation Loss:  0.624, Acc:  0.752, Recall:  0.793, Precision:  0.702,F1 Score:  0.745
Validation loss decreased (inf --> 0.624444). Saving model ...


Validation: 100%|██████████| 33/33 [00:07<00:00,  4.36it/s]


Epoch 2/2
Train Loss:  0.586, Acc:  0.763, Recall:  0.723, Precision:  0.784,F1 Score:  0.752
Validation Loss:  0.524, Acc:  0.792, Recall:  0.772, Precision:  0.772,F1 Score:  0.772
Validation loss decreased (0.624444 --> 0.524454). Saving model ...


{'loss': [0.7024181485176086, 0.5862274169921875],
 'acc': [0.6597499847412109, 0.762583315372467],
 'val_loss': [0.6244438886642456, 0.5244537591934204],
 'val_acc': [0.751893937587738, 0.7916666865348816],
 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: False\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0\n)',
 'loss_func': 'None',
 'epochs': 2,
 'patience': 2}